# Test notebook

In [7]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.july)

In [2]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [3]:
addArchRThreads(1)

Setting default number of Parallel threads to 1.



In [4]:
addArchRLocking(locking = TRUE)

Setting ArchRLocking to TRUE.



In [1]:
load(file = "/home/adufour/work/rds_storage/omics/archr_all_v7_embryo.RData")

In [2]:
colors_dutch <- c("#996ea5", "#ce8787", "#abf5ff", "#78c4ce")

In [3]:
names(colors_dutch) <- c("C1", "C2", "C3", "C4")

In [9]:
# Get additional functions, etc.:
scriptPath <- "/home/adufour/work/scScalpChromatin"
source(paste0(scriptPath, "/plotting_config.R"))
source(paste0(scriptPath, "/misc_helpers.R"))
source(paste0(scriptPath, "/archr_helpers.R"))

Warning message:
“le package ‘ggrastr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘dplyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘tidyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘RcppAlgos’ a été compilé avec la version R 4.2.3”


In [10]:
plotDir <- "/home/adufour/work/notebook/plots/omics/gene_publish"

In [11]:
GImat <- readRDS(file = "/home/adufour/work/rds_storage/omics/RNA_matrix_emb.rds")

# Plot Gene track with RNA expression, peaks gene localisation and peak2gene

In [28]:
gene <- "SREBF2"

p <- plotBrowserTrack(
    ArchRProj = archrproj_sub, 
    groupBy = "Clusters",
    geneSymbol = gene,
    pal = colors_dutch,
    #plotSummary = c("bulkTrack", "featureTrack", "loopTrack", "geneTrack"),
    sizes = c(8, 0.8, 1.25, 0.5),
    #region = plotRegions, 
    loops = getPeak2GeneLinks(archrproj_sub), # All peak-to-gene loops
    tileSize=250,
    minCells=100,
    upstream = 250000,
    downstream = 250000,
    #title = markerGenes
)

archrproj_sub@projectMetadata$outputDirectory <- plotDir

plotPDF(plotList = p, 
    name = paste0(gene, "_tracks_embryos.pdf"), 
    ArchRProj = archrproj_sub, 
    addDOC = FALSE, 
    width = 8, height = 11)

ArchR logging to : ArchRLogs/ArchR-plotBrowserTrack-2d30bc201833c6-Date-2024-02-19_Time-17-00-16.log
If there is an issue, please report to github with logFile!

2024-02-19 17:00:16 : Validating Region, 0.002 mins elapsed.



GRanges object with 1 range and 2 metadata columns:
      seqnames          ranges strand |            gene_id      symbol
         <Rle>       <IRanges>  <Rle> |        <character> <character>
  [1]        5 6719490-6784671      - | ENSSSCG00000026092      SREBF2
  -------
  seqinfo: 263 sequences (1 circular) from an unspecified genome; no seqlengths


2024-02-19 17:00:16 : Adding Bulk Tracks (1 of 1), 0.003 mins elapsed.

Getting Region From Arrow Files 1 of 7

Getting Region From Arrow Files 2 of 7

Getting Region From Arrow Files 3 of 7

Getting Region From Arrow Files 4 of 7

Getting Region From Arrow Files 5 of 7

Getting Region From Arrow Files 6 of 7

Getting Region From Arrow Files 7 of 7

2024-02-19 17:00:29 : Adding Feature Tracks (1 of 1), 0.209 mins elapsed.

2024-02-19 17:00:29 : Adding Loop Tracks (1 of 1), 0.211 mins elapsed.

2024-02-19 17:00:29 : Adding Gene Tracks (1 of 1), 0.217 mins elapsed.

2024-02-19 17:00:30 : Plotting, 0.224 mins elapsed.

ArchR logging successful to : ArchRLogs/ArchR-plotBrowserTrack-2d30bc201833c6-Date-2024-02-19_Time-17-00-16.log

Plotting Gtable!



NULL


In [29]:
markerGenes <- c(gene, "GRHL1")

data_mat <- assays(GImat)[[1]]
rownames(data_mat) <- rowData(GImat)$name
sub_mat <- data_mat[markerGenes,]

grouping_data <- data.frame(cluster=factor(archrproj_sub$Clusters, 
  ordered=TRUE))
rownames(grouping_data) <- getCellNames(archrproj_sub)
sub_mat <- sub_mat[,rownames(grouping_data)]

dodge_width <- 0.75
dodge <- position_dodge(width=dodge_width)

In [30]:
pList <- list()
for(gn in markerGenes){
  df <- data.frame(grouping_data, gene=sub_mat[gn,])
  # Sample to no more than 500 cells per cluster
  set.seed(1)
  df <- df %>% group_by(cluster) %>% dplyr::slice(sample(min(500, n()))) %>% ungroup()
  df <- df %>% as.data.frame()

  covarLabel <- "cluster"  

  # Plot a violin / box plot
  p <- (
    ggplot(df, aes(x=cluster, y=gene, fill=cluster))
    + geom_violin(aes(fill=cluster), adjust = 1.0, scale='width', position=dodge)
    + scale_color_manual(values=colors_dutch, name=covarLabel, na.value="grey")
    + scale_fill_manual(values=colors_dutch)
    + guides(fill=guide_legend(title=covarLabel), 
      colour=guide_legend(override.aes = list(size=5)))
    + ggtitle(gn)
    + xlab("")
    + ylab("RNA Expression")
    + theme_BOR(border=TRUE)
    + theme(panel.grid.major=element_blank(), 
            panel.grid.minor= element_blank(), 
            plot.margin = unit(c(0.25,1,0.25,1), "cm"), 
            legend.position = "none", # Remove legend
            axis.text.x = element_text(angle = 90, hjust = 1)) 
  )
  pList[[gn]] <- p
}

In [31]:
pdf(paste0(plotDir, "/Expression_Violin_embryos_", gene, ".pdf"), width=10, height=4)
pList
dev.off()

$SREBF2

$GRHL1


png 
  2